In [4]:
import os
import torchvision.models as models 
import torch.nn as nn
import torch
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

device0 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device1 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device2 = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

데이터셋

In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_glob, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, names=['file_name', 'label'])
        self.img_dir = img_glob
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

device(type='cuda', index=2)

In [7]:
pd.read_csv('../../data/OriginalData/MMR.csv')

,PathologyNumber,RecriptDate,P53,MLH1,MSH2,MSH6,PMS2,MMR status,"Subtype(1,endometrioid;2,serous;3,clear;4mixed;5etc;6MMMT)"
0,S10-01345,2010-01-18,Wild,0,1,1,0,Defect,1
1,S10-01350,2010-01-18,Wild,1,1,1,1,Normal,1
2,S10-01926,2010-01-25,Null,1,1,1,1,Normal,1
3,S10-03151,2010-02-08,Wild,1,1,1,1,Normal,1
4,S10-06438,2010-03-19,Wild,1,1,1,1,Normal,1
...,...,...,...,...,...,...,...,...,...
337,S22-19730,2022-08-23,Wild,0,1,1,0,Defect,1
338,S22-19946,2022-08-25,Wild,1,1,1,1,Normal,1
339,S22-20342,2022-08-30,Wild,1,1,1,1,Normal,1
340,S22-21614,2022-09-15,Wild,1,1,1,1,Normal,1
